In [8]:
import pandas as pd
import os

In [9]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("aljarah/xAPI-Edu-Data")

# print("Path to dataset files:", path)

In [10]:
df_path = r"C:\Users\Fhuan\.cache\kagglehub\datasets\aljarah\xAPI-Edu-Data\versions\6\xAPI-Edu-Data.csv"
df = pd.read_csv(df_path)
display(df)

,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays,Class
0,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,15,16,2,20,Yes,Good,Under-7,M
1,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,20,20,3,25,Yes,Good,Under-7,M
2,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,10,7,0,30,No,Bad,Above-7,L
3,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,30,25,5,35,No,Bad,Above-7,L
4,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,40,50,12,50,No,Bad,Above-7,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,F,Jordan,Jordan,MiddleSchool,G-08,A,Chemistry,S,Father,5,4,5,8,No,Bad,Above-7,L
476,F,Jordan,Jordan,MiddleSchool,G-08,A,Geology,F,Father,50,77,14,28,No,Bad,Under-7,M
477,F,Jordan,Jordan,MiddleSchool,G-08,A,Geology,S,Father,55,74,25,29,No,Bad,Under-7,M
478,F,Jordan,Jordan,MiddleSchool,G-08,A,History,F,Father,30,17,14,57,No,Bad,Above-7,L


In [11]:
df.columns

Index(['gender', 'NationalITy', 'PlaceofBirth', 'StageID', 'GradeID',
       'SectionID', 'Topic', 'Semester', 'Relation', 'raisedhands',
       'VisITedResources', 'AnnouncementsView', 'Discussion',
       'ParentAnsweringSurvey', 'ParentschoolSatisfaction',
       'StudentAbsenceDays', 'Class'],
      dtype='object')

In [12]:
# Reemplazar valores vacíos por NaN
df.replace(["?", "NA", "None", ""], pd.NA, inplace=True)

# Imputar valores numéricos con la media
num_cols = ['raisedhands','VisITedResources','AnnouncementsView','Discussion']
df[num_cols] = df[num_cols].fillna(df[num_cols].mean())

# Imputar categóricos con la moda
cat_cols = [
    'gender','NationalITy','PlaceofBirth','StageID','GradeID','SectionID',
    'Topic','Semester','Relation','ParentAnsweringSurvey',
    'ParentschoolSatisfaction','StudentAbsenceDays','Class'
]
df[cat_cols] = df[cat_cols].fillna(df[cat_cols].mode().iloc[0])

# Resultado pandas
print("DataFrame procesado con Pandas:")
print(df.head())



# ================================
#     PREPROCESAMIENTO EN SPARK
# ================================
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import (
    StringIndexer, OneHotEncoder, VectorAssembler, MinMaxScaler
)

spark = SparkSession.builder.appName("Preprocessing").getOrCreate()

# Convertir pandas → Spark DataFrame
spark_df = spark.createDataFrame(df)

# --- 1. Codificación de variables categóricas ---
categorical_cols = [
    'gender','NationalITy','PlaceofBirth','StageID','GradeID','SectionID',
    'Topic','Semester','Relation','ParentAnsweringSurvey',
    'ParentschoolSatisfaction','StudentAbsenceDays','Class'
]

indexers = [
    StringIndexer(inputCol=col, outputCol=f"{col}_idx", handleInvalid="keep")
    for col in categorical_cols
]

# One-Hot Encoder
encoders = [
    OneHotEncoder(inputCol=f"{c}_idx", outputCol=f"{c}_oh")
    for c in categorical_cols
]

# --- 2. Columnas numéricas ---
numeric_cols = ['raisedhands','VisITedResources','AnnouncementsView','Discussion']

# Ensamblar características
assembler = VectorAssembler(
    inputCols=[f"{c}_oh" for c in categorical_cols] + numeric_cols,
    outputCol="vector_features"
)

# --- 3. Normalización ---
scaler = MinMaxScaler(inputCol="vector_features", outputCol="scaled_features")



DataFrame procesado con Pandas:
  gender NationalITy PlaceofBirth     StageID GradeID SectionID Topic  \
0      M          KW       KuwaIT  lowerlevel    G-04         A    IT   
1      M          KW       KuwaIT  lowerlevel    G-04         A    IT   
2      M          KW       KuwaIT  lowerlevel    G-04         A    IT   
3      M          KW       KuwaIT  lowerlevel    G-04         A    IT   
4      M          KW       KuwaIT  lowerlevel    G-04         A    IT   

  Semester Relation  raisedhands  VisITedResources  AnnouncementsView  \
0        F   Father           15                16                  2   
1        F   Father           20                20                  3   
2        F   Father           10                 7                  0   
3        F   Father           30                25                  5   
4        F   Father           40                50                 12   

   Discussion ParentAnsweringSurvey ParentschoolSatisfaction  \
0          20             

In [13]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier

# Convertir variable objetivo a numérica
label_indexer = StringIndexer(
    inputCol="Class",
    outputCol="label",
    handleInvalid="keep"
)

# Modelo predictivo
rf = RandomForestClassifier(
    featuresCol="scaled_features",
    labelCol="label",
    numTrees=100,
    maxDepth=6
)

# Crear pipeline completo
pipeline = Pipeline(stages=indexers + encoders + [
    assembler, scaler, label_indexer, rf
])


In [14]:
train_data, test_data = spark_df.randomSplit([0.8, 0.2], seed=42)


In [15]:
model = pipeline.fit(train_data)


Py4JJavaError: An error occurred while calling o183.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 1.0 failed 1 times, most recent failure: Lost task 4.0 in stage 1.0 (TID 12) (MSI executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:252)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:143)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:158)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:178)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:261)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:70)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:374)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:338)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)

In [ ]:
predictions = model.transform(test_data)

predictions.select("label", "prediction", "probability").show(10)


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)

accuracy = evaluator.evaluate(predictions)
print(f"Precisión del modelo: {accuracy:.4f}")


In [ ]:
f1 = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="f1"
).evaluate(predictions)

print(f"F1-Score: {f1:.4f}")


In [ ]:
pred_pd = predictions.select("label", "prediction").toPandas()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(pred_pd["label"], pred_pd["prediction"])

plt.figure()
sns.heatmap(cm, annot=True, fmt="d")
plt.title("Matriz de Confusión")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()


In [ ]:
rf_model = model.stages[-1]

importances = rf_model.featureImportances.toArray()

plt.figure()
plt.bar(range(len(importances)), importances)
plt.title("Importancia de Características")
plt.xlabel("Índice de Variable")
plt.ylabel("Importancia")
plt.show()
rf_model = model.stages[-1]

importances = rf_model.featureImportances.toArray()

plt.figure()
plt.bar(range(len(importances)), importances)
plt.title("Importancia de Características")
plt.xlabel("Índice de Variable")
plt.ylabel("Importancia")
plt.show()


In [ ]:
pred_pd["label"].value_counts().plot(kind="bar")
plt.title("Distribución de Clases Reales")
plt.show()

pred_pd["prediction"].value_counts().plot(kind="bar")
plt.title("Distribución de Clases Predichas")
plt.show()
